<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/turku_era5_cds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium

# The precise City Center coordinates we chose
lat, lon = 60.5, 22.3

# 1. Initialize the map centered on Turku
# We use 'cartodbpositron' for a clean, professional look that highlights markers
m = folium.Map(location=[lat, lon], zoom_start=12, control_scale=True)

# 2. Add a marker for the exact Grid Center
folium.Marker(
    [lat, lon],
    popup="<b>ERA5-Land Grid Center</b><br>Lat: 60.5, Lon: 22.3",
    tooltip="Turku City Center Point",
    icon=folium.Icon(color='darkred', icon='info-sign')
).add_to(m)

# 3. Add a rectangle to show the ERA5-Land Grid Cell (approx 0.1 x 0.1 degrees)
# This represents the actual area the weather data is calculated for.
folium.Rectangle(
    bounds=[[60.45, 22.25], [60.55, 22.35]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    popup="Approximate 9km Grid Cell Area"
).add_to(m)

# 4. Add a reference marker for the Market Square (Kauppatori)
# to see how close it is to our data point.
folium.CircleMarker(
    location=[60.4515, 22.2672],
    radius=5,
    color="green",
    fill=True,
    popup="Market Square (City Center)"
).add_to(m)

# Display the map in Colab
m

In [2]:
! pip install "cdsapi>=0.7.7"

In [3]:
!pip install netcdf4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [4]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

Configuration updated! Now attempting to update the library...


In [5]:
import cdsapi
import os
import calendar

# CRITICAL FIX: Set progress=False to prevent widget metadata creation
c = cdsapi.Client(progress=False)

# Range: Full year 2024 + January 2025
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

print("--- Data Download Started (GitHub-Safe Mode) ---")

for i, (year, month) in enumerate(tasks):
    filename = f'turku_{year}_{month}.nc'

    if os.path.exists(filename):
        print(f"[{i+1}/13] {filename} already exists. Skipping.")
        continue

    # Determine days: Full month for 2024, but only Jan 1st for 2025
    if year == '2025':
        days = ['01']
    else:
        last_day = calendar.monthrange(int(year), int(month))[1]
        days = [str(d).zfill(2) for d in range(1, last_day + 1)]

    print(f"[{i+1}/13] Requesting {month}/{year}...")

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    '2m_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                ],
                'year': year,
                'month': month,
                'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [60.5, 22.3, 60.5, 22.3],
                'format': 'netcdf',
            },
            filename)
        # We print a success message that will stay in your GitHub logs
        print(f"      ✅ {filename} successfully downloaded and saved.")
    except Exception as e:
        print(f"      ❌ Failed {year}-{month}: {e}")

print("--- All Downloads Complete ---")

2026-03-01 08:14:41,632 INFO [2026-02-25T00:00:00Z] Please note that the announced software upgrade for extracting geographical-area data from selected ERA5 and Seasonal Forecast datasets was successfully implemented on 25 February. For further details, please [visit our forum announcement](https://forum.ecmwf.int/t/software-upgrade-for-data-extraction-of-a-geographical-area-from-selected-era5-and-seasonal-forecast-datasets/14583).
INFO:ecmwf.datastores.legacy_client:[2026-02-25T00:00:00Z] Please note that the announced software upgrade for extracting geographical-area data from selected ERA5 and Seasonal Forecast datasets was successfully implemented on 25 February. For further details, please [visit our forum announcement](https://forum.ecmwf.int/t/software-upgrade-for-data-extraction-of-a-geographical-area-from-selected-era5-and-seasonal-forecast-datasets/14583).


--- Data Download Started (GitHub-Safe Mode) ---
[1/13] Requesting 01/2024...


2026-03-01 08:14:41,909 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2026-03-01 08:14:41,912

      ✅ turku_2024_01.nc successfully downloaded and saved.
[2/13] Requesting 02/2024...


2026-03-01 08:23:04,815 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2026-03-01 08:23:04,818

...

      ✅ turku_2024_11.nc successfully downloaded and saved.
[12/13] Requesting 12/2024...


2026-03-01 09:28:56,526 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2026-03-01 09:28:56,529

      ✅ turku_2024_12.nc successfully downloaded and saved.
[13/13] Requesting 01/2025...


2026-03-01 09:35:18,629 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2026-03-01 09:35:18,632

      ✅ turku_2025_01.nc successfully downloaded and saved.
--- All Downloads Complete ---


In [6]:
!pip install netcdf4 h5netcdf

In [10]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import zipfile

# 1. Setup paths
file_list = sorted(glob.glob('turku_*.nc'))
extract_dir = 'extracted_temp'
os.makedirs(extract_dir, exist_ok=True)
all_data = []

print(f"--- Processing {len(file_list)} files ---")

for f in file_list:
    try:
        # ZIP check logic
        with open(f, 'rb') as test_f:
            is_zip = test_f.read(2) == b'PK'

        if is_zip:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            inner_files = glob.glob(os.path.join(extract_dir, "*.nc"))
            if inner_files:
                ds = xr.open_dataset(inner_files[0], engine='netcdf4')
                for extra in inner_files: os.remove(extra)
        else:
            ds = xr.open_dataset(f, engine='netcdf4')

        # --- FIX: Rename 'time' dimension to 'valid_time' for compatibility if it exists ---
        if 'time' in ds.dims:
            ds = ds.rename({'time': 'valid_time'})

        # --- CRITICAL FIX: GHI DE-ACCUMULATION ---
        # ERA5-Land ssrd is accumulated from 00:00 UTC.
        # We subtract the previous hour from the current hour to get Joules per that specific hour.
        ssrd_diff = ds['ssrd'].diff(dim='valid_time', label='upper')
        first_hour = ds['ssrd'].isel(valid_time=0)

        # Recombine: First hour + the differenced hours
        ghi_joules = xr.concat([first_hour, ssrd_diff], dim='valid_time')

        # Convert to DataFrame
        df = ds.to_dataframe().reset_index()

        # Apply conversions
        df['GHI_Wm2'] = ghi_joules.values.flatten() / 3600
        df['Temperature_C'] = df['t2m'] - 273.15
        df['Wind_Speed_ms'] = np.sqrt(df['u10']**2 + df['v10']**2)

        all_data.append(df)
        ds.close()

        print(f"✅ Processed: {f}")
    except Exception as e:
        print(f"❌ Error on {f}: {e}")

if all_data:
    df_combined = pd.concat(all_data).sort_values('valid_time')

    # 2. Format Time Columns
    df_combined['valid_time'] = pd.to_datetime(df_combined['valid_time'])
    df_combined['Date'] = df_combined['valid_time'].dt.strftime('%Y-%m-%d')
    df_combined['Hour'] = df_combined['valid_time'].dt.strftime('%H:00')

    # 3. Final selection and reorder
    final_cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2']
    final_output = df_combined[final_cols]

    # 4. Export
    csv_name = 'Turku_Weather_Hourly_2024_Clean.csv'
    final_output.to_csv(csv_name, index=False)

    print("\n" + "="*40)
    print(f"SUCCESS: Saved to {csv_name}")
    print(f"Total Rows: {len(final_output)}")
    print("="*40)

    # Display preview
    print("\nData Preview (check if GHI_Wm2 looks realistic, e.g., 0 at night):")
    print(final_output.head(25)) # Showing 25 rows to see a full day cycle
else:
    print("No data frames found.")

--- Processing 13 files ---
✅ Processed: turku_2024_01.nc
✅ Processed: turku_2024_02.nc
✅ Processed: turku_2024_03.nc
✅ Processed: turku_2024_04.nc
✅ Processed: turku_2024_05.nc
✅ Processed: turku_2024_06.nc
✅ Processed: turku_2024_07.nc
✅ Processed: turku_2024_08.nc
✅ Processed: turku_2024_09.nc
✅ Processed: turku_2024_10.nc
✅ Processed: turku_2024_11.nc
✅ Processed: turku_2024_12.nc
✅ Processed: turku_2025_01.nc

SUCCESS: Saved to Turku_Weather_Hourly_2024_Clean.csv
Total Rows: 8808

Data Preview (check if GHI_Wm2 looks realistic, e.g., 0 at night):
          Date   Hour  Temperature_C  Wind_Speed_ms     GHI_Wm2
0   2024-01-01  00:00     -14.486420       2.236743  169.474442
1   2024-01-01  01:00     -14.128998       2.165980 -169.474442
2   2024-01-01  02:00     -13.787689       2.090343    0.000000
3   2024-01-01  03:00     -13.063324       2.009852    0.000000
4   2024-01-01  04:00     -12.227142       2.373250    0.000000
5   2024-01-01  05:00     -11.744965       2.263318    0.0

In [13]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import zipfile

# 1. Setup paths
file_list = sorted(glob.glob('turku_*.nc'))
extract_dir = 'extracted_temp'
os.makedirs(extract_dir, exist_ok=True)
all_data = []

print(f"--- Processing {len(file_list)} files ---")

for f in file_list:
    try:
        # ZIP check logic
        with open(f, 'rb') as test_f:
            is_zip = test_f.read(2) == b'PK'

        if is_zip:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            inner_files = glob.glob(os.path.join(extract_dir, "*.nc"))
            if inner_files:
                ds = xr.open_dataset(inner_files[0], engine='netcdf4')
                for extra in inner_files: os.remove(extra)
        else:
            ds = xr.open_dataset(f, engine='netcdf4')

        # --- DYNAMIC DIMENSION DETECTION ---
        # Detect if the time dimension is 'time' or 'valid_time'
        t_dim = 'valid_time' if 'valid_time' in ds.dims else 'time'

        # --- GHI DE-ACCUMULATION ---
        # Using the detected dimension name
        ssrd_diff = ds['ssrd'].diff(dim=t_dim, label='upper')
        first_hour = ds['ssrd'].isel({t_dim: 0})

        # Recombine: First hour + the differenced hours
        ghi_joules = xr.concat([first_hour, ssrd_diff], dim=t_dim)

        # Convert to DataFrame
        df = ds.to_dataframe().reset_index()

        # Apply conversions (Flatten to ensure 1D array matches DF length)
        df['GHI_Wm2'] = ghi_joules.values.flatten() / 3600
        #FIX: Clip negative values to zero
        df['GHI_Wm2'] = df['GHI_Wm2'].clip(lower=0)

        df['Temperature_C'] = df['t2m'] - 273.15
        df['Wind_Speed_ms'] = np.sqrt(df['u10']**2 + df['v10']**2)

        all_data.append(df)
        ds.close()

        print(f"✅ Processed: {f}")
    except Exception as e:
        print(f"❌ Error on {f}: {e}")

if all_data:
    df_combined = pd.concat(all_data)

    # Standardize time column name for the final CSV
    time_col = 'valid_time' if 'valid_time' in df_combined.columns else 'time'
    df_combined = df_combined.sort_values(time_col)

    # 2. Format Time Columns
    df_combined[time_col] = pd.to_datetime(df_combined[time_col])
    df_combined['Date'] = df_combined[time_col].dt.strftime('%Y-%m-%d')
    df_combined['Hour'] = df_combined[time_col].dt.strftime('%H:00')

    # 3. Final selection and reorder
    final_cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2']
    final_output = df_combined[final_cols]

    # 4. Export
    csv_name = 'Turku_Weather_Hourly_Final.csv'
    final_output.to_csv(csv_name, index=False)

    print("\n" + "="*40)
    print(f"SUCCESS: Saved to {csv_name}")
    print(f"Total Rows: {len(final_output)}")
    print("="*40)

    # Display preview
    print("\nData Preview:")
    print(final_output.head(10))
else:
    print("No data frames found. Check if the files are corrupted or empty.")

--- Processing 13 files ---
✅ Processed: turku_2024_01.nc
✅ Processed: turku_2024_02.nc
✅ Processed: turku_2024_03.nc
✅ Processed: turku_2024_04.nc
✅ Processed: turku_2024_05.nc
✅ Processed: turku_2024_06.nc
✅ Processed: turku_2024_07.nc
✅ Processed: turku_2024_08.nc
✅ Processed: turku_2024_09.nc
✅ Processed: turku_2024_10.nc
✅ Processed: turku_2024_11.nc
✅ Processed: turku_2024_12.nc
✅ Processed: turku_2025_01.nc

SUCCESS: Saved to Turku_Weather_Hourly_Final.csv
Total Rows: 8808

Data Preview:
         Date   Hour  Temperature_C  Wind_Speed_ms     GHI_Wm2
0  2024-01-01  00:00     -14.486420       2.236743  169.474442
1  2024-01-01  01:00     -14.128998       2.165980    0.000000
2  2024-01-01  02:00     -13.787689       2.090343    0.000000
3  2024-01-01  03:00     -13.063324       2.009852    0.000000
4  2024-01-01  04:00     -12.227142       2.373250    0.000000
5  2024-01-01  05:00     -11.744965       2.263318    0.000000
6  2024-01-01  06:00     -11.416595       2.430776    0.000